In [135]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [271]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Layer, Dense, Flatten, Concatenate, Reshape, Add
from tensorflow.keras import Input, Model
from tensorflow import pad, Variable
from tensorflow.keras.initializers import HeUniform

import argparse
import numpy as np
import pennylane as qml

from qml_hep_lhc.data import QuarkGluon, MNIST
from qml_hep_lhc.models.base_model import BaseModel
from qml_hep_lhc.layers.utils import normalize_padding, normalize_tuple, convolution_iters

import tensorflow as tf
tf.keras.backend.set_floatx('float32')

In [272]:
args = argparse.Namespace()
args.center_crop = 0.7
args.resize = [8,8]
args.standardize = 1
args.binary_data = [0,1]
args.optimizer = 'Ranger'
args.labels_to_categorical = 1
args.processed = 1
args.percent_samples = 0.1
args.batch_size = 4

In [273]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Center cropping...
Center cropping...
Resizing data...
Resizing data...
Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤═══════════════╤═══════════════╤═══════════════╤═══════════╕
│ Data   │ Train size    │ Val size      │ Test size     │ Dims      │
╞════════╪═══════════════╪═══════════════╪═══════════════╪═══════════╡
│ X      │ (48, 8, 8, 1) │ (12, 8, 8, 1) │ (20, 8, 8, 1) │ (8, 8, 1) │
├────────┼───────────────┼───────────────┼───────────────┼───────────┤
│ y      │ (48, 2)       │ (12, 2)       │ (20, 2)       │ (2,)      │
╘════════╧═══════════════╧═══════════════╧═══════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════╡
│ Train Images │ -1.58 │  7.68 │  -0.01 │  0.98 │ [

In [274]:
# LR Scheduler callback
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                          factor=0.1,
                                          patience=5,
                                          min_delta=0.0001,
                                          min_lr=1e-6)
callbacks = [lr_scheduler_callback]

## Defining QNode

In [275]:
# Hyperparameters

n_qubits = 3
n_layers = 2
n_inputs = 9

args.learning_rate = 0.1
args.template = 'S2D'
args.cluster_state = 1
args.n_qubits = n_qubits
args.n_layers = n_layers 
args.epochs = 2

In [276]:
import pennylane as qml


dev = qml.device("default.qubit", wires=n_qubits)
qubits = list(range(n_qubits))



@qml.qnode(dev, interface='tf')
def qnode_sparse(inputs, w, b):

    # shape of inputs is (n_inputs,)
    # shape of w is (n_layers, n_qubits, 3, n_inputs)
    # shape of b is (n_layers, n_qubits, 3)
    
    z = tf.tensordot(w, inputs, axes = 1) + b
    
    for l in range(n_layers):
        for q in qubits:
            qml.Rot(z[l,q,0], z[l,q,1], z[l,q,2], wires= q)
        
        if (l & 1):
            for q0, q1 in zip(qubits[1::2], qubits[2::2] + [qubits[0]]):
                qml.CZ((q0,q1))

        else:
            for q0, q1 in zip(qubits[0::2], qubits[1::2]):
                qml.CZ((q0,q1))
                
    return [qml.expval(qml.PauliZ(q)) for q in qubits]

@qml.qnode(dev, interface='tf')
def qnode_sd(inputs,w,b):
    z = tf.tensordot(w, inputs, axes = 1) + b
    qml.SimplifiedTwoDesign(initial_layer_weights=tf.zeros((n_qubits,)), weights=z, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(q)) for q in qubits]

In [277]:
inputs = tf.random.uniform((n_inputs, ))
template = args.template
shapes = get_node(template, n_layers, n_qubits, n_inputs)
w = tf.random.uniform(shapes['w'])
b = tf.random.uniform(shapes['b'])
out = tf.tensordot(w,tf.transpose(inputs),axes = 1) + b
out.shape

TensorShape([2, 2, 2])

In [278]:
drawer = qml.draw(qnode_sd, expansion_strategy="device")
print(drawer(inputs,w,b))

0: ──RY(0.00)─╭●──RY(3.57)──────────────╭●──RY(1.75)──────────────┤  <Z>
1: ──RY(0.00)─╰Z──RY(2.48)─╭●──RY(3.94)─╰Z──RY(2.96)─╭●──RY(2.03)─┤  <Z>
2: ──RY(0.00)──────────────╰Z──RY(3.37)──────────────╰Z──RY(2.26)─┤  <Z>


In [279]:
qnode_sd(inputs,w,b)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([0.56440674, 0.68330616, 0.44777675])>

In [280]:
def get_node(template, num_layers, num_qubits, num_inputs):
    if template == 'NQubitPQC':
        return {
            "w": (num_layers, num_qubits, 3, num_inputs),
            "b": (num_layers, num_qubits, 3)
        }
    elif template == 'S2D':
        return {
            "w": (num_layers, num_qubits -1, 2, num_inputs),
            "b": (num_layers, num_qubits -1, 2)
        }

In [281]:
class QConv2D(Layer):
    """
    2D Quantum convolution layer (e.g. spatial convolution over images).
    This layer creates a convolution kernel that is convolved 
    with the layer input to produce a tensor of outputs. Finally,
    `activation` is applied to the outputs as well.
    """

    def __init__(
            self,
            filters=1,
            kernel_size=(3, 3),
            strides=(1, 1),
            n_qubits=1,
            n_layers=1,
            template = 'NQubitPQC',
            padding='valid',
            cluster_state=False,
            observable=None,
            name='QConv2D',
    ):

        super(QConv2D, self).__init__(name=name)

        # Filters
        if isinstance(filters, float):
            filters = int(filters)
        if filters is not None and filters <= 0:
            raise ValueError('Invalid value for argument `filters`. '
                             'Expected a strictly positive value. '
                             f'Received filters={filters}.')
        self.filters = filters

        # Num layers
        if isinstance(n_layers, float):
            n_layers = int(n_layers)
        if n_layers is not None and n_layers <= 0:
            raise ValueError('Invalid value for argument `n_layers`. '
                             'Expected a strictly positive value. '
                             f'Received n_layers={n_layers}.')
        self.n_layers = n_layers

        self.observable = observable
        self.kernel_size = normalize_tuple(kernel_size, 'kernel_size')
        self.strides = normalize_tuple(strides, 'strides')
        self.padding = normalize_padding(padding)
        self.cluster_state = cluster_state
        self.n_qubits = n_qubits
        
        self.n_inputs = np.prod(self.kernel_size)
        self.weight_shapes = get_node(template, self.n_layers, self.n_qubits, self.n_inputs)
        
        
    def build(self, input_shape):
    
        self.iters, self.padding_constant = convolution_iters(
            input_shape[1:3], self.kernel_size, self.strides, self.padding)
        self.n_channels = input_shape[3]

        self.conv_pqcs = [[(filter, channel)
                           for channel in range(self.n_channels)]
                          for filter in range(self.filters)]

        for filter in range(self.filters):
            for channel in range(self.n_channels):
                name = f"{self.name}_{filter}_{channel}"
                self.conv_pqcs[filter][channel] = qml.qnn.KerasLayer(qnode = qnode_sd, 
                                                                     weight_shapes = self.weight_shapes, 
                                                                     output_dim= self.n_qubits
                                                                     )

    def _convolution(self, input_tensor, filter, channel):

        s = self.strides
        k = self.kernel_size

        conv_out = []
        for i in range(self.iters[0]):
            for j in range(self.iters[1]):
                x = input_tensor[:, i * s[0]:i * s[0] + k[0], j *
                                 s[1]:j * s[1] + k[1]]
                x = Flatten()(x)
                conv_out += [self.conv_pqcs[filter][channel](x)]
                
        conv_out = Concatenate(axis=1)(conv_out)
        conv_out = Reshape((self.iters[0], self.iters[1],self.n_qubits ))(conv_out)
        return conv_out

    def call(self, input_tensor):
        input_tensor = pad(input_tensor, self.padding_constant)

        if self.n_channels == 1:
            conv_out = [
                self._convolution(input_tensor[:, :, :, 0], filter, 0)
                for filter in range(self.filters)
            ]
        else:
            conv_out = [
                Add()([
                    self._convolution(input_tensor[:, :, :, c], filter, c)
                    for c in range(self.n_channels)
                ])
                for filter in range(self.filters)
            ]
        conv_out = Concatenate(axis=-1)(conv_out)
        return conv_out

In [282]:
class QCNN(BaseModel):
    """
	General Quantum Convolutional Neural Network
	"""

    def __init__(self, data_config, args=None):
        super(QCNN, self).__init__(args)
        self.args = vars(args) if args is not None else {}

        # Data config
        self.input_dim = data_config["input_dims"]
        self.cluster_state = self.args.get("cluster_state", False)
        self.n_layers = self.args.get("n_layers", 1)
        self.n_qubits = self.args.get("n_qubits", 1)
        self.template = self.args.get("template", 'NQubitPQC')

        input_shape = [None] + list(self.input_dim)

        self.qconv2d_1 = QConv2D(
            filters=1,
            kernel_size=3,
            strides=2,
            n_qubits=self.n_qubits,
            n_layers=self.n_layers,
            template = self.template,
            padding="same",
            cluster_state=self.cluster_state,
            name='qconv2d_1',
        )
        
        self.dense1 = Dense(8, activation='relu')
        self.dense2 = Dense(2, activation='softmax')
    
    def call(self, input_tensor):
        x = self.qconv2d_1(input_tensor)
        x = Flatten()(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

    def build_graph(self):
        x = Input(shape=self.input_dim, batch_size= 4)
        return Model(inputs=[x],
                     outputs=self.call(x),
                     name=f"QCNN-{self.fm_class}-{self.ansatz_class}")

    @staticmethod
    def add_to_argparse(parser):
        parser.add_argument("--cluster-state",
                            action="store_true",
                            default=False)
        parser.add_argument("--n-layers", type=int, default=1)
        parser.add_argument("--n-qubits", type=int, default=1)
        parser.add_argument("--template",type=str,  default='NQubitPQC')
        return parser

In [283]:
model = QCNN(data.config(),args)

Using Ranger optimizer


In [284]:
model.compile()

In [285]:
model.fit(data, callbacks)

Epoch 1/2
12/12 [==============================] - 57s 5s/step - loss: 0.7298 - auc_25: 0.5382 - accuracy: 0.5417 - val_loss: 0.5583 - val_auc_25: 0.8056 - val_accuracy: 0.8333 - lr: 0.1000
Epoch 2/2
12/12 [==============================] - 62s 5s/step - loss: 0.4653 - auc_25: 0.9701 - accuracy: 0.8750 - val_loss: 0.3957 - val_auc_25: 0.9792 - val_accuracy: 0.9167 - lr: 0.1000
